In [ ]:
#pip install imparaai-checkers

#### Import Packages and Define Functions

In [1]:
from checkers.game import Game
import random
import numpy as np
import tensorflow as tf
import pandas as pd

In [12]:
def generateModel():
    #defines model1 which is 3-layer neural network with reLu activation function
    model = tf.keras.Sequential(name="model")
    model.add(tf.keras.Input(shape=(256, 256, 3)))
    model.add(tf.keras.layers.Conv2D(16, 5, activation="sigmoid", padding="same"))
    model.add(tf.keras.layers.Conv2D(32, 5, activation="sigmoid", padding="same"))
    model.add(tf.keras.layers.Conv2D(8, 5, activation="sigmoid", padding="same"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding="valid"))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(96, activation="sigmoid"))
    model.add(tf.keras.layers.Dense(3, activation="softmax"))
    
    model.compile()
    model.summary()
    return model



In [13]:
generateModel()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 256, 256, 16)      1216      
                                                                 
 conv2d_10 (Conv2D)          (None, 256, 256, 32)      12832     
                                                                 
 conv2d_11 (Conv2D)          (None, 256, 256, 8)       6408      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 128, 128, 8)      0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 131072)            0         
                                                                 
 dense_8 (Dense)             (None, 96)                12583008  
                                                             

In [21]:
def convertLocation(position):
    location = [int((position - 1)/4), 2*((position-1)%4)]
    if (location[0] %2==0):
        location[1] += 1
    return location

def convertBoard(game):
    board = np.zeros((8,8))
    for piece in game.board.pieces:
        if piece.position == None:
            continue
        location = [int((piece.position-1)/4), 2*((piece.position-1)%4)]
        if (location[0] %2==0):
            location[1] += 1
        if piece.player == 1 and piece.king:
            board[location[0], location[1]] = -2
        elif piece.player == 1:
            board[location[0], location[1]] = -1
        elif piece.player == 2 and piece.king:
            board[location[0], location[1]] = 2
        else:
            board[location[0], location[1]] = 1
    return board

def printBoard(board):
    length =64
    chars = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    print(" "*4 + "*"*length)
    print("\t 0\t 1\t 2\t 3\t 4\t 5\t 6\t 7\n")
    print(" " * 4 + "-"*length)
    for i in range(8):
        print(f"{i}||", end="")
        for j in range(8):
            if board[i,j] == 0:
                print("\t ", end="")
            else:
                print(f"\t {board[i,j]}", end="")
        print()
        print("\n" +" "* 4 + "-"*length)
    print(" " * 4 +"-"*length+"\n\n")

def boardMove(board, move):
    nextBoard = np.copy(board)
    startLocation = convertLocation(move[0])
    endLocation = convertLocation(move[1])
    nextBoard[endLocation[0], endLocation[1]] = nextBoard[startLocation[0], startLocation[1]]
    nextBoard[startLocation[0], startLocation[1]] = 0
    return nextBoard
    
def generateAllNextBoards(game, board):
    movesList = game.get_possible_moves()
    nextBoardList = []
    for move in movesList:
        nextBoardList.append(boardMove(board, move))
    return nextBoardList

def switchBoard(board):
    newBoard = np.flip(board, (0,1))
    newBoard = newBoard * -1
    return newBoard
    
def getBestMove(model, game, player=1):
    board = convertBoard(game)
    if player==2:
        board = switchBoard(board)
    nextBoardList = generateAllNextBoards(game, board)
    for count, boardOption in enumerate(nextBoardList):
        boardOption = boardOption.reshape(1,8,8)
        value = model(boardOption).numpy()[0][0]
        if count == 0:
            bestMove = (value, 0)
        elif bestMove[0] < value:
            bestMove = (value, count)
    return bestMove[1]

In [3]:
def generateModel():
    #defines model1 which is 3-layer neural network with reLu activation function
    model = tf.keras.Sequential(name="model")
    model.add(tf.keras.Input(shape=(8, 8, 1)))
    model.add(tf.keras.layers.Conv2D(3, 5, activation="sigmoid"))
    model.add(tf.keras.layers.Conv2D(3, 3, activation="sigmoid"))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(6, activation="sigmoid"))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    
    model.compile()
    #model.summary()
    return model

def updateWeights(model, weights):
    layer0Weights = weights[0:75].reshape((5,5,1,3))
    layer0bias = weights[75:78]
    layer1Weights = weights[78:159].reshape((3,3,3,3))
    layer1bias = weights[159:162]
    layer3Weights = weights[162:234].reshape((12,6))
    layer3bias = weights[234:240]
    layer4Weights = weights[240:246].reshape((6,1))
    layer4bias = weights[246].reshape((1,))
    
    orderedWeights = [[layer0Weights, layer0bias], [layer1Weights, layer1bias], [], [layer3Weights, layer3bias], [layer4Weights, layer4bias]]
    
    for i, layer in enumerate(model.layers):
        if i == 2:
            continue
        layer.set_weights(orderedWeights[i])
        


In [4]:
def playGame(model1, model2):
    game = Game()
    count = 0
    while(not game.is_over()):
    #for i in range(3):
        player = game.whose_turn()
        movesList = game.get_possible_moves()
        if player == 1:
            #print("Player move 1: ")
            moveInput = getBestMove(model1, game)
        else:
            #print("Player 2 moving: ")
            moveInput = getBestMove(model1, game, player=2)
        moveChoice = movesList[int(moveInput)]
        #print(f"Moving pawn at position {moveChoice[0]} to position {moveChoice[1]}")
        game.move(moveChoice)
        #if count % 5 == 0:
            #rintBoard(convertBoard(game))
        count = count + 1

    #print(game.get_winner())
    return game.get_winner()

def singleElimination(df, locations = None):
    if locations is None:
        number = df.shape[0]
        indexes = list(range(number))

    else:
        number = len(locations)
        indexes = locations
        
    random.shuffle(indexes)
    outputSize = int(number/2)
    
    winners = []
    ties = []
    randomCount = 0
    
    model1 = generateModel()
    model2 = generateModel()
    
    while len(indexes) > 1:
        index1 = indexes.pop()
        index2 = indexes.pop()
        weights1 = df.iloc[index1,:].to_numpy()
        weights2 = df.iloc[index2,:].to_numpy()
        updateWeights(model1, weights1)
        updateWeights(model2, weights2)
        
        winner = playGame(model1, model2)
        
        if winner is None:
            ties.append(index1)
            ties.append(index2)
        elif winner == 1:
            winners.append(index1)
        elif winner == 2:
            winners.append(index2)
        else:
            ties.append(index1)
            ties.append(index2)
    
    while len(ties) > 1:
        if len(indexes) > 0:
            index1 = indexes.pop()
            index2 = ties.pop()
        else:
            index1 = ties.pop()
            index2 = ties.pop()
        
        weights1 = df.iloc[index1,:].to_numpy()
        weights2 = df.iloc[index2,:].to_numpy()
        updateWeights(model1, weights1)
        updateWeights(model2, weights2)
        
        winner = playGame(model1, model2)
        
        if winner is None:
            randomCount += 1
            if random.randint(1,2) == 1:
                winners.append(index1)
            else:
                winners.append(index2)
        elif winner == 1:
            winners.append(index1)
        elif winner == 2:
            winners.append(index2)
        else:
            randomCount += 1
            if random.randint(1,2) == 1:
                winners.append(index1)
            else:
                winners.append(index2)
    if len(winners) == outputSize:
        print(f'number of randoms = {randomCount} out of {outputSize}')
        winners.sort()
        return winners
    else:
        print(f'winners size {len(winners)} != expected size {outputSize}')
        return 0


def RoundRobin4(df, locations=None):
    number = df.shape[0]
    
    model1 = generateModel()
    model2 = generateModel()
    
    if locations is None:
        indexes = list(range(number))
    else:
        indexes = locations
        
    numberOfGroups = int(len(indexes)/4)
    
    random.shuffle(indexes)
    nextRound = []
    
    groups = []
    for i in range(numberOfGroups):
        group = [indexes.pop(), indexes.pop(), indexes.pop(), indexes.pop()]
        groups.append(group)
        
    games = [[0,1], [2,0], [0,3], [1,2], [1,3], [3,2]]
    
    for group in groups:
        RR_df = pd.DataFrame({ "IndexNum" : group, "points" : 0, "wins" : 0})
        for game in games:
            index1 = group[game[0]]
            index2 = group[game[1]]
            
            weights1 = df.iloc[index1,:].to_numpy()
            weights2 = df.iloc[index2,:].to_numpy()
            updateWeights(model1, weights1)
            updateWeights(model2, weights2)
            
            winner = playGame(model1, model2)
            if winner is None:
                RR_df.iloc[game[0], 1] += 1
                RR_df.iloc[game[1], 1] += 1
            elif winner == 1:
                RR_df.iloc[game[0], 1] += 3
                RR_df.iloc[game[0], 2] += 1
            elif winner == 2:
                RR_df.iloc[game[1], 1] += 3
                RR_df.iloc[game[1], 2] += 1
            else:
                RR_df.iloc[game[0], 1] += 1
                RR_df.iloc[game[1], 1] += 1
        
        bestIndex = RR_df.nlargest(2, ["points", "wins"])
        nextRound.extend(bestIndex["IndexNum"].tolist())
    
    return nextRound

#input is trimmed df
def RoundRobin16(df):
    
    indexes = list(range(16))
    
    model1 = generateModel()
    model2 = generateModel()
      
    games = []
    for i in range(15):
        for j in range(i+1, 16):
            x = [i,j]
            random.shuffle(x)
            games.append(x)
    
    RR_df = pd.DataFrame({ "names" : df.index, "points" : 0, "wins" : 0})
    for game in games:
        index1 = game[0]
        index2 = game[1]

        weights1 = df.iloc[index1,:].to_numpy()
        weights2 = df.iloc[index2,:].to_numpy()
        updateWeights(model1, weights1)
        updateWeights(model2, weights2)

        winner = playGame(model1, model2)
        if winner is None:
            RR_df.iloc[index1, 1] += 1
            RR_df.iloc[index2, 1] += 1
        elif winner == 1:
            RR_df.iloc[index1, 1] += 3
            RR_df.iloc[index1, 2] += 1
        elif winner == 2:
            RR_df.iloc[index2, 1] += 3
            RR_df.iloc[index2, 2] += 1
        else:
            RR_df.iloc[index1, 1] += 1
            RR_df.iloc[index2, 1] += 1
        
        
    rankingsDF = RR_df.sort_values(by=['points', 'wins'], ascending=False)
    rankingsList = rankingsDF["names"].tolist()
    
    return rankingsDF, rankingsList
            

In [5]:
def recombination(df, gen):
    count = 0
    pairs = []
    for i in range(15):
        for j in range(i+1, 16):
            x = [i,j]
            pairs.append(x)
    for pair in pairs:
        w1 = df.iloc[pair[0]].to_numpy().copy()
        w2 = df.iloc[pair[1]].to_numpy().copy()
        pos = random.randint(0,246)
        w1_1 = w1[:pos]
        w1_2 = w1[pos:]
        w2_1 = w2[:pos]
        w2_2 = w2[pos:]
        set1 = np.concatenate((w1_1, w2_2), axis=None)
        set2 = np.concatenate((w2_1, w1_2), axis=None)
        df.loc[f'Gen{gen}_{count:04d}'] = set1
        count += 1
        df.loc[f'Gen{gen}_{count:04d}'] = set2
        count += 1
        
    return df, count

def mutations(df, count, gen):
    for index, row in df.iterrows():
        for i in range(3):
            newRow = row.copy() + np.random.uniform(-.1,.1,247)
            df.loc[f'Gen{gen}_{count:04d}'] = newRow
            count += 1
    return df
 


In [6]:
#need function to take rankings and/or points to create next generation
#bind selection and generation into one function
def selection(inputPath):
    weightsDF = pd.read_csv(inputPath, index_col='names')
    R1Winners = singleElimination(weightsDF)
    R2Winners = singleElimination(weightsDF, locations=R1Winners)
    R3Winners = singleElimination(weightsDF, locations=R2Winners)
    
    weightsDF1 = weightsDF.iloc[R3Winners, :]
    RR1 = RoundRobin4(weightsDF1)
    RR2 = RoundRobin4(weightsDF1, locations=RR1)
    RR3 = RoundRobin4(weightsDF1, locations=RR2)
    
    RR3.sort()
    return weightsDF.iloc[RR3, :]

def generation(df, gen, outputPath=None):
    if outputPath is None:
        outputPath = f'Gen{gen}Weights.csv'
    nextGen, count = recombination(df, gen)
    nextGen = mutations(nextGen, count, gen)
    
    nextGen.to_csv(outputPath)
    print(f'Gen{gen} created')

#### Generate 1024 Gen0 models

In [67]:
path = 'gen0Weights.csv'
location = 0
count = 0

columnName = []
for i in range(247):
    columnName.append("w"+str(i))
weightsDF = pd.DataFrame(columns=columnName)
 
while count < 1024:
    weights1 = np.random.uniform(-1,1,247)
    model1 = generateModel()
    updateWeights(model1, weights1)

    weights2 = np.random.uniform(-1,1,247)
    model2 = generateModel()
    updateWeights(model2, weights2)

    bothWeights = [0, weights1, weights2]

    winner = playGame(model1, model2)
    if winner is None:
        continue
    elif winner == 1:
        weightsDF.loc[count] = weights2
        count += 1
    elif winner == 2:
        weightsDF.loc[count] = weights1
        count += 1
    else:
        continue
    
    if count % 10 == 0:
        print(count)

    
weightsDF.to_csv(path)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020


#### Selection of gen0 and generation of gen1

In [46]:
path = 'gen0Weights.csv'
weightsDF = pd.read_csv(path, index_col='names')

In [57]:
R1Winners = singleElimination(weightsDF)
print(R1Winners)
R2Winners = singleElimination(weightsDF, locations=R1Winners)
R3Winners = singleElimination(weightsDF, locations=R2Winners)
print(R3Winners)



number of randoms = 25 out of 512
[0, 1, 2, 6, 7, 9, 10, 12, 13, 17, 18, 19, 21, 22, 23, 25, 26, 28, 30, 33, 35, 37, 38, 41, 43, 48, 49, 51, 52, 53, 54, 56, 61, 63, 65, 66, 68, 69, 72, 75, 77, 78, 79, 80, 81, 83, 84, 87, 88, 89, 90, 91, 92, 95, 101, 103, 105, 106, 107, 110, 111, 113, 116, 117, 120, 122, 125, 126, 128, 131, 132, 136, 139, 143, 144, 148, 149, 152, 153, 154, 155, 156, 157, 158, 160, 163, 166, 167, 168, 170, 171, 172, 179, 182, 184, 185, 187, 190, 191, 200, 201, 203, 204, 205, 207, 211, 212, 215, 216, 219, 220, 221, 222, 224, 225, 226, 229, 230, 234, 235, 236, 237, 239, 242, 243, 249, 252, 256, 257, 259, 260, 261, 264, 266, 271, 273, 275, 277, 278, 280, 282, 285, 287, 288, 289, 291, 292, 293, 294, 295, 296, 300, 301, 303, 304, 305, 307, 308, 310, 315, 318, 319, 320, 324, 326, 330, 331, 333, 334, 335, 336, 337, 341, 343, 344, 346, 347, 351, 354, 355, 359, 360, 363, 364, 365, 366, 367, 369, 370, 376, 377, 379, 380, 381, 382, 383, 384, 385, 386, 387, 389, 392, 393, 400, 401, 

In [63]:
weightsDF1 = weightsDF.iloc[R3Winners, :]
#print(weightsDF1.describe())
path = 'gen0.1Weights.csv'
weightsDF1.to_csv(path)

In [5]:
path = 'gen0.1Weights.csv'
weightsDF = pd.read_csv(path, index_col='names')

In [6]:
RR1 = RoundRobin4(weightsDF)
RR2 = RoundRobin4(weightsDF, locations=RR1)
RR3 = RoundRobin4(weightsDF, locations=RR2)
RR3.sort()
unrankedWeights = weightsDF.iloc[RR3, :]

In [8]:
path = 'gen0.2Weights.csv'
unrankedWeights.to_csv(path)

In [34]:
df = pd.read_csv(path, index_col='names')
df, count = recombination(df)
df = mutations(df, count)
df.to_csv('Gen1Weights.csv')

In [43]:
nextGen = mutations(weightsDF, 240, 2)
nextGen.to_csv('Gen2Weights.csv')

#### After making gen1, created and used selection and generation function to generate next generation

In [7]:
weightsDF = selection('Gen12Weights.csv')
generation(weightsDF, 13)

weightsDF = selection('Gen13Weights.csv')
generation(weightsDF, 14)

number of randoms = 103 out of 512
number of randoms = 51 out of 256
number of randoms = 10 out of 128
Gen13 created
number of randoms = 99 out of 512
number of randoms = 63 out of 256
number of randoms = 14 out of 128
Gen14 created


In [8]:
weightsDF = selection('Gen14Weights.csv')
generation(weightsDF, 15)

weightsDF = selection('Gen15Weights.csv')
rankedDF, rankedList = RoundRobin16(weightsDF)
print(rankedDF)
generation(weightsDF, 16)

number of randoms = 119 out of 512
number of randoms = 83 out of 256
number of randoms = 18 out of 128
Gen15 created
number of randoms = 116 out of 512
number of randoms = 70 out of 256
number of randoms = 17 out of 128
         names  points  wins
15  Gen15_0111      32     9
12  Gen15_0086      27     6
5   Gen15_0010      19     2
11  Gen15_0085      19     2
6   Gen15_0015      17     1
0   Gen14_0041      16     1
1   Gen14_0055      16     1
4   Gen15_0006      16     1
8   Gen15_0053      16     1
13  Gen15_0087      16     1
2   Gen14_0061      15     1
14  Gen15_0094      15     1
10  Gen15_0075      15     0
9   Gen15_0070      14     0
3   Gen14_0104       9     0
7   Gen15_0027       6     1
Gen16 created


#### Testing the best Model Against Best of Gen 0

In [16]:
weightsDF = pd.read_csv('Gen15Weights.csv', index_col='names')
model1 = generateModel()
bestWeights = weightsDF.loc['Gen15_0111',:].to_numpy()
updateWeights(model1, bestWeights)
weightsDF2 = pd.read_csv('gen0.2Weights.csv', index_col='names')
model2 = generateModel()
record = [0,0,0]
record2 = [0,0,0]
for i in range(16):
    weights = weightsDF2.iloc[i].to_numpy()
    updateWeights(model2, weights)
    winner = playGame(model1, model2)
    if winner == 1:
        record[0] += 1
    elif winner is None:
        record[1] += 1
    elif winner == 2:
        record[2] += 1
    
    winner = playGame(model2, model1)
    if winner == 1:
        record2[0] += 1
    elif winner is None:
        record2[1] += 1
    elif winner == 2:
        record2[2] += 1

print(f'Wins\tTies\tLosses\n{record[0]}\t{record[1]}\t{record[2]}\n{record2[0]}\t{record[1]}\t{record[2]}')

Wins	Ties	Losses
16	0	0
16	0	0


#### Test the best model against best of gen 5

In [19]:
weightsDF = pd.read_csv('Gen15Weights.csv', index_col='names')
model1 = generateModel()
bestWeights = weightsDF.loc['Gen15_0111',:].to_numpy()
updateWeights(model1, bestWeights)
weightsDF2 = pd.read_csv('gen5Weights.csv', index_col='names')
model2 = generateModel()
record = [0,0,0]
record2 = [0,0,0]
for i in range(16):
    weights = weightsDF2.iloc[i].to_numpy()
    updateWeights(model2, weights)
    winner = playGame(model1, model2)
    if winner == 1:
        record[0] += 1
    elif winner is None:
        record[1] += 1
    elif winner == 2:
        record[2] += 1
    
    winner = playGame(model2, model1)
    if winner == 1:
        record2[0] += 1
    elif winner is None:
        record2[1] += 1
    elif winner == 2:
        record2[2] += 1


print(f'Wins\tTies\tLosses\n{record[0]}\t{record[1]}\t{record[2]}\n{record2[0]}\t{record2[1]}\t{record2[2]}')

Wins	Ties	Losses
16	0	0
8	4	4


#### Test the best model against best of gen 10

In [20]:
weightsDF = pd.read_csv('Gen15Weights.csv', index_col='names')
model1 = generateModel()
bestWeights = weightsDF.loc['Gen15_0111',:].to_numpy()
updateWeights(model1, bestWeights)
weightsDF2 = pd.read_csv('gen10Weights.csv', index_col='names')
model2 = generateModel()
record = [0,0,0]
record2 = [0,0,0]
for i in range(16):
    weights = weightsDF2.iloc[i].to_numpy()
    updateWeights(model2, weights)
    winner = playGame(model1, model2)
    if winner == 1:
        record[0] += 1
    elif winner is None:
        record[1] += 1
    elif winner == 2:
        record[2] += 1
    
    winner = playGame(model2, model1)
    if winner == 1:
        record2[0] += 1
    elif winner is None:
        record2[1] += 1
    elif winner == 2:
        record2[2] += 1


print(f'Wins\tTies\tLosses\n{record[0]}\t{record[1]}\t{record[2]}\n{record2[0]}\t{record2[1]}\t{record2[2]}')

Wins	Ties	Losses
16	0	0
6	6	4


#### Play against the best AI

In [22]:
weightsDF = pd.read_csv('Gen15Weights.csv', index_col='names')
model = generateModel()
bestWeights = weightsDF.loc['Gen15_0111',:].to_numpy()
updateWeights(model, bestWeights)

game = Game()
while(not game.is_over()):
#for i in range(3):
    player = game.whose_turn()
    movesList = game.get_possible_moves()
    if player == 2:
        print("Player 2 Pick a move: ")
        movesList = game.get_possible_moves()
        printBoard(convertBoard(game))
        for number, move in enumerate(movesList):
            start = move[0]
            end = move[1]
            print(f"{number}: {convertLocation(start)}, {convertLocation(end)}")
        print("\nEnter move number:\n ", end="")
        moveInput = input()
    else:
        print("Player 1 moving: ")
        moveInput = getBestMove(model, game)
    moveChoice = movesList[int(moveInput)]
    print(f"Moving pawn at position {moveChoice[0]} to position {moveChoice[1]}")
    game.move(moveChoice)

print(game.get_winner())

Player 1 moving: 
Moving pawn at position 9 to position 14
Player 2 Pick a move: 
    ****************************************************************
	 0	 1	 2	 3	 4	 5	 6	 7

    ----------------------------------------------------------------
0||	 	 -1.0	 	 -1.0	 	 -1.0	 	 -1.0

    ----------------------------------------------------------------
1||	 -1.0	 	 -1.0	 	 -1.0	 	 -1.0	 

    ----------------------------------------------------------------
2||	 	 	 	 -1.0	 	 -1.0	 	 -1.0

    ----------------------------------------------------------------
3||	 	 	 -1.0	 	 	 	 	 

    ----------------------------------------------------------------
4||	 	 	 	 	 	 	 	 

    ----------------------------------------------------------------
5||	 1.0	 	 1.0	 	 1.0	 	 1.0	 

    ----------------------------------------------------------------
6||	 	 1.0	 	 1.0	 	 1.0	 	 1.0

    ----------------------------------------------------------------
7||	 1.0	 	 1.0	 	 1.0	 	 1.0	 

    ---------------

 0
Moving pawn at position 18 to position 9
Player 1 moving: 
Moving pawn at position 5 to position 14
Player 2 Pick a move: 
    ****************************************************************
	 0	 1	 2	 3	 4	 5	 6	 7

    ----------------------------------------------------------------
0||	 	 -1.0	 	 -1.0	 	 -1.0	 	 -1.0

    ----------------------------------------------------------------
1||	 	 	 -1.0	 	 -1.0	 	 	 

    ----------------------------------------------------------------
2||	 	 	 	 -1.0	 	 -1.0	 	 

    ----------------------------------------------------------------
3||	 1.0	 	 -1.0	 	 	 	 	 

    ----------------------------------------------------------------
4||	 	 	 	 	 	 	 	 1.0

    ----------------------------------------------------------------
5||	 1.0	 	 1.0	 	 	 	 	 

    ----------------------------------------------------------------
6||	 	 	 	 1.0	 	 	 	 1.0

    ----------------------------------------------------------------
7||	 1.0	 	 1.0	 	 1.0	 	 

 0
Moving pawn at position 19 to position 10
Player 2 Pick a move: 
    ****************************************************************
	 0	 1	 2	 3	 4	 5	 6	 7

    ----------------------------------------------------------------
0||	 	 	 	 -1.0	 	 	 	 -1.0

    ----------------------------------------------------------------
1||	 -1.0	 	 -1.0	 	 -1.0	 	 	 

    ----------------------------------------------------------------
2||	 	 	 	 1.0	 	 	 	 1.0

    ----------------------------------------------------------------
3||	 1.0	 	 -1.0	 	 	 	 	 

    ----------------------------------------------------------------
4||	 	 	 	 	 	 	 	 

    ----------------------------------------------------------------
5||	 1.0	 	 1.0	 	 	 	 	 

    ----------------------------------------------------------------
6||	 	 	 	 	 	 	 	 

    ----------------------------------------------------------------
7||	 1.0	 	 1.0	 	 1.0	 	 1.0	 

    --------------------------------------------------------------

 1
Moving pawn at position 6 to position 2
Player 1 moving: 
Moving pawn at position 8 to position 11
Player 2 Pick a move: 
    ****************************************************************
	 0	 1	 2	 3	 4	 5	 6	 7

    ----------------------------------------------------------------
0||	 	 2.0	 	 2.0	 	 	 	 

    ----------------------------------------------------------------
1||	 	 	 	 	 	 	 	 

    ----------------------------------------------------------------
2||	 	 	 	 	 	 -1.0	 	 

    ----------------------------------------------------------------
3||	 	 	 -1.0	 	 	 	 	 

    ----------------------------------------------------------------
4||	 	 	 	 	 	 2.0	 	 

    ----------------------------------------------------------------
5||	 1.0	 	 1.0	 	 	 	 	 

    ----------------------------------------------------------------
6||	 	 	 	 	 	 	 	 

    ----------------------------------------------------------------
7||	 1.0	 	 1.0	 	 1.0	 	 1.0	 

    ---------------------